# Imports

In [ ]:
import itertools as it
from pathlib import Path

import dask
import distributed
import h5py
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
from cytoolz import partial
from dask_jobqueue import SLURMCluster
from distributed import Client
from IPython.display import Video
from PIL import Image, ImageDraw, ImageFont
from tqdm.auto import tqdm, trange

from paulssonlab.util.ui import display_image

In [ ]:
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

In [ ]:
from paulssonlab.image_analysis import mosaic, workflow

# Config

In [ ]:
nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220704/220704rbs_library_fish.nd2"
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220718/RBS_DEG_library_20x.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(nd2_filename)

In [ ]:
# default_channels = ("YFP-DUAL", "RFP-Penta")
default_channels = ("YFP-DUAL", "RFP-PENTA")
channel_to_color = {
    "BF": "#ffffff",
    "RFP-PENTA": "#e22400",
    "RFP-Penta": "#e22400",
    "YFP-DUAL": "#f5eb00",
    # "GFP": "#76ba40",
    "Cy5": "#e292fe",
    # "Cy7": "#FF0000"
    # "BFP": "#3a87fd",
}

In [ ]:
font = ImageFont.truetype("fira/FiraSans-Medium.ttf")

In [ ]:
dask.config.set({"distributed.scheduler.allowed-failures": 10})

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="4GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster.scale(50)

In [ ]:
cluster.adapt(maximum=300)

In [ ]:
cluster

# Mosaic

In [ ]:
%%time
extrema = mosaic.get_intensity_extrema(nd2, ("YFP-DUAL", "RFP-PENTA"))

In [ ]:
extrema

In [ ]:
scaling_funcs = mosaic.get_scaling_funcs(extrema)
# scaling_funcs = get_scaling_funcs({"YFP-DUAL": (262, 8000), "RFP-Penta": (278, 8000)})

In [ ]:
%%time
num_t = nd2.sizes["t"]
# scale = [80,40,20,10,5,1]#it.repeat(80)
# scale = np.geomspace(80, 0.1, 900)
# scale = np.geomspace(0.5, 0.1, 60)
scale = [40]
# scale = [0.333]
# scale = it.repeat(0.1)
# scale = it.repeat(0.4)
# scale = np.geomspace(20, 0.05, num_t*3)# + np.linspace(0, 0, 0) #it.repeat(0.3)  # [0.3, 0.3, 0.3, 0.3]
# timepoints = range(0, 119, 30)
# timepoints = range(0, 110, 10)#[20,40,60]
timepoints = it.repeat(60)  # [20]  # [20,40,60,80]
# timepoints = range(num_t)
# timepoints = it.chain(range(num_t), range(num_t), range(num_t)) #[20]#[20, 40, 60]
offset = [2000, 1000]  # [0,0]#np.array([604, 354])
animation_delayed = mosaic.mosaic_animate_scale(
    nd2_filename,
    scale,
    timepoints=timepoints,
    scaling_funcs=scaling_funcs,
    offset=offset,
    channels=default_channels,
    channel_to_color=channel_to_color,
    overlay_func=partial(
        mosaic.square_overlay,
        min_scale=80,
        min_n=0,
        min_width=0.5,
        max_scale=0.1,
        max_n=5,
        max_width=0.9,
        font=font,
    ),
    delayed=True,
)

In [ ]:
%%time
a = dask.compute(animation_delayed, scheduler="sync")[0]

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(a[0])

In [ ]:
%%time
# animation_future = client.compute(animation_delayed)
animation_future = [client.compute(a) for a in tqdm(animation_delayed)]

In [ ]:
%%time
a = client.gather(animation_future)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(a[-1])

In [ ]:
import cv2

In [ ]:
x = a[0].copy()

In [ ]:
ft = cv2.freetype.createFreeType2()
ft.loadFontData(fontFileName="fira/FiraSans-Medium.ttf", id=0)
# y = np.zeros_like(x, dtype=np.uint8)
y = np.zeros((800, 600, 3), dtype=np.uint8)
ft.putText(
    img=y,
    text="abc",
    org=(15, 15),
    fontHeight=101,
    color=(255, 100, 200),
    thickness=-1,
    line_type=cv2.LINE_AA,
    bottomLeftOrigin=False,
)
plt.imshow(y)

In [ ]:
np.unique(y[:, :, 0] - y[:, :, 1])

In [ ]:
y[:, :, 1]

In [ ]:
plt.scatter(y[:, :, 0], y[:, :, 1])

In [ ]:
np.unique(y)

In [ ]:
x.shape

In [ ]:
cv2.putText(x, "", "")

In [ ]:
import cairo

In [ ]:
data = numpy.ndarray(shape=(height, width), dtype=numpy.uint32)
surface = cairo.ImageSurface.create_for_data(data, cairo.FORMAT_ARGB32, width, height)
cr = cairo.Context(surface)

cr.scale(200, 100)
cr.set_line_width(0.04)

cr.select_font_face("Sans", cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_BOLD)
cr.set_font_size(0.35)

cr.move_to(0.04, 0.53)
cr.show_text("Hello")

cr.move_to(0.27, 0.65)
cr.text_path("void")
cr.set_source_rgb(0.5, 0.5, 1)
cr.fill_preserve()
cr.set_source_rgb(0, 0, 0)
cr.set_line_width(0.01)
cr.stroke()

# draw helping lines
cr.set_source_rgba(1, 0.2, 0.2, 0.6)
cr.arc(0.04, 0.53, 0.02, 0, 2 * np.pi)
cr.arc(0.27, 0.65, 0.02, 0, 2 * np.pi)
cr.fill()

In [ ]:
?surface.map_to_image

In [ ]:
z

In [ ]:
np.asarray(z)

In [ ]:
plt.imshow(z)

In [ ]:
x = np.zeros((1024, 512, 3))
y = mosaic.square_overlay(x, 1, 8.5, font=font)
plt.figure(figsize=(20, 20))
plt.imshow(y)

In [ ]:
%%time
mosaic.export_video(
    a,
    "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_zoomtest_4.mp4",
    fps=10,
)

In [ ]:
Video(
    "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_3x3_5fps.mp4",
    embed=True,
)

# FISH from ND2

In [ ]:
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/230103/230103rfp1_500ms.nd2"
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/230103/230103gfp4_500ms.nd2"
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/230105/230105gfp4_500ms.nd2"
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/230130/230130_lane3_200ms.nd2"
nd2_filename = (
    "/home/jqs1/scratch/jqs1/microscopy/230130/230130_lane4_200ms_freshfov001.nd2"
)
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/230131/230131_200ms.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(nd2_filename)

In [ ]:
nd2.sizes

In [ ]:
def get_nd2_frame(filename, v, channel, t):
    # nd2 = workflow.get_nd2_reader(filename)
    nd2 = nd2reader.ND2Reader(filename)
    channels = [c for c in nd2.metadata["channels"] if channel in c]
    if len(channels) != 1:
        raise ValueError(f"found multiple matching channels: {channels}")
    return nd2.get_frame_2D(v=v, t=t, c=nd2.metadata["channels"].index(channels[0]))

In [ ]:
get_nd2_frame(nd2_filename, 0, "RFP", 0)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_nd2_frame,
    nd2_filename,
    0,
    0,
    ["Cy5"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

# FISH

In [ ]:
# fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/220704/FISH/real_run")
# fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/220718/FISH/real_run")
# fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/221206/FISH/real_run")
# fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/230111/FISH/real_run")
fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/230213/FISH/real_run")
# fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/230215/FISH/real_run")

In [ ]:
from functools import lru_cache


@lru_cache
def get_fish_frame(filename, v, channel, t):
    with h5py.File(filename / f"fov={v}_config={channel}_t={t}") as f:
        frame = f["data"][()]
    return frame


def colorized_frame(
    colors, get_frame_func, filename, v, t, channels, scaling_funcs=None
):
    if scaling_funcs:
        imgs = [
            scaling_funcs[channel](get_frame_func(filename, v, channel, t))
            for channel in channels
        ]
    else:
        imgs = [get_frame_func(filename, v, channel, t) for channel in channels]
    colors = [colors[channel] for channel in channels]
    return mosaic.colorize(imgs, colors, scale=(not scaling_funcs))

In [ ]:
# fish_colors = {
#     "BF": "#ffffff",
#     "RFP": "#e22400",
#     # "YFP-DUAL": "#f5eb00",
#     "GFP": "#00ff31",
#     "Cy5": "#e292fe",
#     "Cy7": "#00faff"
#     # "BFP": "#3a87fd",
# }

fish_colors = {
    "BF": "#ffffff",
    "GFP": "#f44336",
    "Cy5": "#03a9f4",
    # "Cy7": "#ffeb3b"
    "Cy7": "#8bc34a",
}

In [ ]:
# 220704
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "RFP": (200, 4000),
#         "Cy5": (100, 13000),
#         "Cy7": (500, 4_000),
#     }
# )
# 220718
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "RFP": (5_000, 22_000),
#         "Cy5": (8_000, 25000),
#         "Cy7": (2_900, 3_600),
#     }
# )
# 220718 (tight, use this)
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (30_000, 65_500),
#         "RFP": (8_000, 10_000),
#         "Cy5": (13_000, 17_000),
#         #"Cy7": (2_900, 3_050),
#         "Cy7": (2_950, 3_250),
#     }
# )
# 221206
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "RFP": (400, 16000),
#         "GFP": (5_000, 40_500),
#         "Cy5": (100, 10_000),
#         "Cy7": (200, 3_000),
#     }
# )
# 230111
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "RFP": (400, 16000),
#         "GFP": (5_000, 60_500),
#         "Cy5": (100, 30_000),
#         "Cy7": (200, 3_000),
#     }
# )
# 230130
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "RFP": (400, 16000),
#         "GFP": (2_000, 30_500),
#         "Cy5": (100, 10_000),
#         "Cy7": (200, 1_000),
#     }
# )
# 230213/230215 (causes clipping [white] issues)
fish_scaling_funcs = mosaic.get_scaling_funcs(
    {
        "BF": (40_000, 65_500),
        "GFP": (10_000, 15_000),
        "Cy5": (1_000, 9_000),
        "Cy7": (700, 1_500),
    }
)
# 230213/230215 (looser)
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "GFP": (10_000, 18_000),
#         "Cy5": (1_000, 10_000),
#         "Cy7": (700, 2_000),
#     }
# )
# old
# fish_scaling_funcs = mosaic.get_scaling_funcs(
#     {
#         "BF": (40_000, 65_500),
#         "GFP": (40_000, 65_500),
#         "RFP": (8000, 15000),
#         "Cy5": (5_000, 40000),
#         "Cy7": (2500, 4_000),
#     }
# )

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    7,
    9,
    ["Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
display_image(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    7,
    9,
    ["GFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
display_image(a)

In [ ]:
%%time
frames = [
    colorized_frame(
        fish_colors,
        get_fish_frame,
        fish_dir,
        7,
        t,
        ["Cy7"],  # ["GFP", "Cy5", "Cy7"],
        scaling_funcs=fish_scaling_funcs,
    )
    for t in trange(1, 11)
]

In [ ]:
import skimage.transform

In [ ]:
%%time
rescaled_frames = [
    skimage.transform.rescale(f, 0.5, anti_aliasing=True, channel_axis=-1)
    for f in tqdm(frames)
]

In [ ]:
%%time
mosaic.export_video(rescaled_frames, "/home/jqs1/_temp/230213_FISH_fov7_Cy7.mp4", fps=5)

In [ ]:
%%time
mosaic.export_video(frames, "/home/jqs1/_temp/230213_FISH_fov7_full.mp4", fps=5)

In [ ]:
!du -hs /home/jqs1/_temp/220718_FISH_fov33.mp4